Admin Zero to Snowflake walkthrough

1. Start by setting up a schema for use throughout the Notebook
2. Establish access and functional roles, assigning them to a service and person user
3. Confirm correct access of new roles
4. Explore different queries that look at role privileges and user roles

In [ ]:
SET WAREHOUSE_NAME ='COMPUTE_WH';

USE ROLE ACCOUNTADMIN;
USE WAREHOUSE IDENTIFIER($WAREHOUSE_NAME);

CREATE OR REPLACE DATABASE Z2S;

CREATE OR REPLACE SCHEMA Z2S.DEMO_FINANCE;

CREATE OR REPLACE TABLE Z2S.DEMO_FINANCE.REVENUE_DATA (
    REGION VARCHAR(50),
    QUARTER VARCHAR(10), 
    REVENUE DECIMAL(15,2),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

INSERT INTO Z2S.DEMO_FINANCE.REVENUE_DATA VALUES 
('North America', 'Q1 2024', 2500000.00, CURRENT_TIMESTAMP()),
('Europe', 'Q1 2024', 1800000.00, CURRENT_TIMESTAMP()),
('Asia Pacific', 'Q1 2024', 2200000.00, CURRENT_TIMESTAMP());

CREATE OR REPLACE SCHEMA Z2S.DEMO_HR;
CREATE OR REPLACE TABLE Z2S.DEMO_HR.EMPLOYEE_INFO (
    EMPLOYEE_ID INT,
    FULL_NAME VARCHAR(100),
    DEPARTMENT VARCHAR(50),
    SALARY DECIMAL(10,2),
    HIRE_DATE DATE
);

INSERT INTO Z2S.DEMO_HR.EMPLOYEE_INFO VALUES 
(1001, 'Sarah Johnson', 'Finance', 85000.00, '2023-01-15'),
(1002, 'Mike Chen', 'Engineering', 95000.00, '2023-03-20'),
(1003, 'Lisa Rodriguez', 'HR', 78000.00, '2023-02-10');

CREATE OR REPLACE TABLE Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE (
    REGION VARCHAR(50),
    QUARTER VARCHAR(10), 
    REVENUE DECIMAL(15,2),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

INSERT INTO Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE VALUES 
('North America', 'Q1 2024', 2500000.00, CURRENT_TIMESTAMP()),
('Europe', 'Q1 2024', 1800000.00, CURRENT_TIMESTAMP()),
('Asia Pacific', 'Q1 2024', 2200000.00, CURRENT_TIMESTAMP());

CREATE OR REPLACE TABLE Z2S.DEMO_FINANCE.WAREHOUSE_CACHE_EXAMPLE (
    REGION VARCHAR(50),
    QUARTER VARCHAR(10), 
    REVENUE DECIMAL(15,2),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

INSERT INTO Z2S.DEMO_FINANCE.WAREHOUSE_CACHE_EXAMPLE VALUES 
('North America', 'Q1 2024', 2500000.00, CURRENT_TIMESTAMP()),
('Europe', 'Q1 2024', 1800000.00, CURRENT_TIMESTAMP()),
('Asia Pacific', 'Q1 2024', 2200000.00, CURRENT_TIMESTAMP());

In [ ]:
CREATE OR REPLACE DATABASE ROLE Z2S_DB_R;
CREATE OR REPLACE DATABASE ROLE Z2S_DB_W;
CREATE OR REPLACE DATABASE ROLE Z2S_DB_C;

In [ ]:
-- SETUP READ ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_FIN_R;

GRANT USAGE, MONITOR  ON DATABASE Z2S  TO DATABASE ROLE Z2S_SCH_FIN_R;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_R;

GRANT SELECT ON ALL TABLES IN SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_R;
GRANT SELECT ON FUTURE TABLES IN SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_R;

-- SETUP WRITE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_FIN_W;

GRANT INSERT, UPDATE, DELETE, TRUNCATE ON ALL TABLES IN SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_W;
GRANT INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE TABLES IN SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_W;

-- SETUP CREATE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_FIN_C;

GRANT USAGE, MONITOR  ON DATABASE Z2S  TO DATABASE ROLE Z2S_SCH_FIN_C;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_C;

GRANT CREATE TABLE ON SCHEMA Z2S.DEMO_FINANCE TO DATABASE ROLE Z2S_SCH_FIN_C;

-- SETUP INHERITANCE: CREATE -> WRITE -> READ
GRANT DATABASE ROLE Z2S_SCH_FIN_R TO DATABASE ROLE Z2S_SCH_FIN_W;
GRANT DATABASE ROLE Z2S_SCH_FIN_W TO DATABASE ROLE Z2S_SCH_FIN_C;

-- TIE BACK TO DB ROLES
GRANT DATABASE ROLE Z2S_SCH_FIN_R TO DATABASE ROLE Z2S_DB_R;
GRANT DATABASE ROLE Z2S_SCH_FIN_W TO DATABASE ROLE Z2S_DB_W;
GRANT DATABASE ROLE Z2S_SCH_FIN_C TO DATABASE ROLE Z2S_DB_C;

In [ ]:
-- SETUP READ ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_HR_R;

GRANT USAGE, MONITOR  ON DATABASE Z2S  TO DATABASE ROLE Z2S_SCH_HR_R;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_R;

GRANT SELECT ON ALL TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_R;
GRANT SELECT ON FUTURE TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_R;

-- SETUP WRITE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_HR_W;

GRANT INSERT, UPDATE, DELETE, TRUNCATE ON ALL TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_W;
GRANT INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_W;

-- SETUP CREATE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_HR_C;

GRANT USAGE, MONITOR  ON DATABASE Z2S  TO DATABASE ROLE Z2S_SCH_HR_C;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_C;

GRANT CREATE TABLE ON SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_HR_C;

-- SETUP INHERITANCE: CREATE -> WRITE -> READ
GRANT DATABASE ROLE Z2S_SCH_HR_R TO DATABASE ROLE Z2S_SCH_HR_W;
GRANT DATABASE ROLE Z2S_SCH_HR_W TO DATABASE ROLE Z2S_SCH_HR_C;

-- TIE BACK TO DB ROLES
GRANT DATABASE ROLE Z2S_SCH_HR_R TO DATABASE ROLE Z2S_DB_R;
GRANT DATABASE ROLE Z2S_SCH_HR_W TO DATABASE ROLE Z2S_DB_W;
GRANT DATABASE ROLE Z2S_SCH_HR_C TO DATABASE ROLE Z2S_DB_C;

In [ ]:
CREATE OR REPLACE SCHEMA Z2S.GOVERNANCE;

-- SETUP READ ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_GOVERNANCE_R;

GRANT USAGE, MONITOR  ON DATABASE Z2S TO DATABASE ROLE Z2S_SCH_GOVERNANCE_R;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.GOVERNANCE TO DATABASE ROLE Z2S_SCH_GOVERNANCE_R;

GRANT SELECT ON ALL TABLES IN SCHEMA Z2S.GOVERNANCE TO DATABASE ROLE Z2S_SCH_GOVERNANCE_R;
GRANT SELECT ON FUTURE TABLES IN SCHEMA Z2S.GOVERNANCE TO DATABASE ROLE Z2S_SCH_GOVERNANCE_R;

-- SETUP WRITE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_GOVERNANCE_W;

GRANT INSERT, UPDATE, DELETE, TRUNCATE ON ALL TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_GOVERNANCE_W;
GRANT INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE TABLES IN SCHEMA Z2S.DEMO_HR TO DATABASE ROLE Z2S_SCH_GOVERNANCE_W;

-- SETUP CREATE ACCESS
CREATE OR REPLACE DATABASE ROLE Z2S_SCH_GOVERNANCE_C;

GRANT USAGE, MONITOR  ON DATABASE Z2S TO DATABASE ROLE Z2S_SCH_GOVERNANCE_C;
GRANT USAGE, MONITOR  ON SCHEMA Z2S.GOVERNANCE TO DATABASE ROLE Z2S_SCH_GOVERNANCE_C;

GRANT CREATE TABLE ON SCHEMA Z2S.GOVERNANCE TO DATABASE ROLE Z2S_SCH_GOVERNANCE_C;

-- SETUP INHERITANCE: CREATE -> WRITE -> READ
GRANT DATABASE ROLE Z2S_SCH_GOVERNANCE_R TO DATABASE ROLE Z2S_SCH_GOVERNANCE_W;
GRANT DATABASE ROLE Z2S_SCH_GOVERNANCE_W TO DATABASE ROLE Z2S_SCH_GOVERNANCE_C;

-- TIE BACK TO DB ROLES
GRANT DATABASE ROLE Z2S_SCH_GOVERNANCE_R TO DATABASE ROLE Z2S_DB_R;
GRANT DATABASE ROLE Z2S_SCH_GOVERNANCE_W TO DATABASE ROLE Z2S_DB_W;
GRANT DATABASE ROLE Z2S_SCH_GOVERNANCE_C TO DATABASE ROLE Z2S_DB_C;

In [ ]:
CREATE OR REPLACE ROLE Z2S_FINANCE_ANALYST
COMMENT = 'Functional Role for finance team members - access to finance and analytics data';

-- GIVE CREATE, WRITE, AND READ ACCESS TO FIN SCHEMA
GRANT DATABASE ROLE Z2S_SCH_FIN_C, Z2S_SCH_GOVERNANCE_R TO ROLE Z2S_FINANCE_ANALYST;

CREATE OR REPLACE ROLE Z2S_HR_MANAGER
COMMENT = 'Functional Role for HR team - access to HR and analytics data';

-- GIVE READ ACCESS TO HR SCHEMA
GRANT DATABASE ROLE Z2S_SCH_HR_R, Z2S_SCH_GOVERNANCE_R TO ROLE Z2S_HR_MANAGER;

GRANT ROLE Z2S_FINANCE_ANALYST, Z2S_HR_MANAGER TO ROLE SYSADMIN;

In [ ]:
-- Person User: john_analyst (human user for finance team)
CREATE OR REPLACE USER JOHN_ANALYST
LOGIN_NAME = 'john_analyst'
PASSWORD = 'TempPassword123!'
DEFAULT_ROLE = FINANCE_ANALYST
DEFAULT_WAREHOUSE = COMPUTE_WH
COMMENT = 'Person user - Finance analyst John Smith'
TYPE = PERSON
MUST_CHANGE_PASSWORD = TRUE;

-- Service User: hr_service_account (automated system/API access)  
CREATE OR REPLACE USER HR_SERVICE_ACCOUNT
COMMENT = 'Service user - HR reporting automation'
LOGIN_NAME = 'hr_service_account'
TYPE = SERVICE
RSA_PUBLIC_KEY='-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA855hcbWsi0i8guEOZ79P
Acl/FFUQAu2F+W/gnnZNB09u+9PPl7SDG5Fa72WhlAyGeFh0ZiGPm0zWcOHw3JSP
hkwMP4jgdKB9xcIVNUtSC3l4a70EF32dEdZLab63jJSp6UN9FYX4SzTXyIkTyB2V
Uuxf04naxzLUUGKuZWQiH9v5WCTQUN/MXNVHSVL75HMBCk5ysLDX776KyrKJGE9H
/bA1WtPHSe7/6Iy4rhsnLDXkHey+niUdwuqZjVUXxIH+/2zY98EwxFt9xxYszZSf
I8X6OBARJIWd4/6ykK+2TQ5MvjgXXeOwvbemA//bic+RXhkXbzAvOfi1lJrC3rbL
swIDAQAB
-----END PUBLIC KEY-----';

In [ ]:
GRANT ROLE Z2S_FINANCE_ANALYST TO USER JOHN_ANALYST;
GRANT ROLE Z2S_HR_MANAGER TO USER HR_SERVICE_ACCOUNT;

In [ ]:
USE ROLE Z2S_FINANCE_ANALYST;

SELECT 'ANALYST ACCESS TEST' AS test_type, REGION, QUARTER, REVENUE
FROM Z2S.DEMO_FINANCE.REVENUE_DATA;

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
SHOW GRANTS TO ROLE Z2S_FINANCE_ANALYST;
-- SHOW GRANTS TO USER JOHN_ANALYST;

In [ ]:
SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
  WHERE granted_to = 'USER';

1. Setup Schema for storing Governance related Tags and Policies
2. Create Tags and Assign them to Columns
3. Setup Column Security
4. Setup Row Access Policies with Table Reference
5. Confirm Policy Application

In [ ]:
CREATE OR REPLACE TABLE Z2S.DEMO_HR.EMPLOYEES_PII (
    employee_id NUMBER(10,0),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    ssn VARCHAR(11),
    salary NUMBER(10,2),
    department VARCHAR(50),
    region VARCHAR(20),
    hire_date DATE,
    manager_id NUMBER(10,0),
    phone VARCHAR(15)
);

INSERT INTO Z2S.DEMO_HR.EMPLOYEES_PII VALUES
(1001, 'John', 'Smith', 'john.smith@company.com', '123-45-6789', 85000.00, 'HR', 'NORTH', '2020-01-15', NULL, '555-0101'),
(1002, 'Sarah', 'Johnson', 'sarah.johnson@company.com', '234-56-7890', 92000.00, 'SALES', 'SOUTH', '2019-03-20', NULL, '555-0102'),
(1003, 'Mike', 'Brown', 'mike.brown@company.com', '345-67-8901', 78000.00, 'HR', 'EAST', '2021-06-10', 1001, '555-0103'),
(1004, 'Lisa', 'Davis', 'lisa.davis@company.com', '456-78-9012', 105000.00, 'SALES', 'WEST', '2018-11-05', 1002, '555-0104'),
(1005, 'Tom', 'Wilson', 'tom.wilson@company.com', '567-89-0123', 67000.00, 'HR', 'CENTRAL', '2022-02-28', 1001, '555-0105'),
(1006, 'Amy', 'Taylor', 'amy.taylor@company.com', '678-90-1234', 89000.00, 'SALES', 'NORTH', '2020-08-12', 1002, '555-0106'),
(1007, 'David', 'Miller', 'david.miller@company.com', '789-01-2345', 72000.00, 'HR', 'SOUTH', '2021-12-03', 1001, '555-0107'),
(1008, 'Jennifer', 'Garcia', 'jennifer.garcia@company.com', '890-12-3456', 98000.00, 'SALES', 'EAST', '2019-07-18', 1002, '555-0108');

CREATE OR REPLACE TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII (
    customer_id NUMBER(10,0),
    company_name VARCHAR(100),
    contact_first_name VARCHAR(50),
    contact_last_name VARCHAR(50),
    contact_email VARCHAR(100),
    contact_phone VARCHAR(15),
    region VARCHAR(20),
    annual_revenue NUMBER(15,2),
    created_date DATE,
    account_manager_id NUMBER(10,0)
);

INSERT INTO Z2S.DEMO_FINANCE.CUSTOMERS_PII VALUES
(2001, 'Tech Corp Inc', 'Robert', 'Anderson', 'r.anderson@techcorp.com', '555-1001', 'NORTH', 2500000.00, '2020-01-10', 1002),
(2002, 'Global Systems LLC', 'Maria', 'Rodriguez', 'm.rodriguez@globalsys.com', '555-1002', 'SOUTH', 1800000.00, '2020-03-15', 1004),
(2003, 'Innovation Labs', 'James', 'Thompson', 'j.thompson@innovlabs.com', '555-1003', 'EAST', 3200000.00, '2019-11-20', 1006),
(2004, 'Future Solutions', 'Linda', 'White', 'l.white@futuresol.com', '555-1004', 'WEST', 1200000.00, '2021-02-05', 1008),
(2005, 'Smart Technologies', 'Christopher', 'Lee', 'c.lee@smarttech.com', '555-1005', 'CENTRAL', 2100000.00, '2020-09-12', 1002);

In [ ]:
CREATE TAG IF NOT EXISTS Z2S.GOVERNANCE.PII_LEVEL
  ALLOWED_VALUES 'HIGH', 'MEDIUM', 'LOW';

CREATE TAG IF NOT EXISTS Z2S.GOVERNANCE.DEPARTMENT
  ALLOWED_VALUES 'HR', 'SALES', 'FINANCE', 'IT';

In [ ]:
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII SET TAG Z2S.GOVERNANCE.DEPARTMENT = 'HR';
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII ALTER COLUMN SSN SET TAG Z2S.GOVERNANCE.PII_LEVEL = 'HIGH';
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII ALTER COLUMN EMAIL SET TAG Z2S.GOVERNANCE.PII_LEVEL = 'LOW';
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII ALTER COLUMN SALARY SET TAG Z2S.GOVERNANCE.PII_LEVEL = 'HIGH';

ALTER TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII SET TAG Z2S.GOVERNANCE.DEPARTMENT = 'SALES';
ALTER TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII ALTER COLUMN CONTACT_EMAIL SET TAG Z2S.GOVERNANCE.PII_LEVEL = 'MEDIUM';

In [ ]:
CREATE OR REPLACE MASKING POLICY Z2S.GOVERNANCE.PII_MASK AS (val string) RETURNS string ->
  CASE
    WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN') THEN val
    WHEN SYSTEM$GET_TAG_ON_CURRENT_COLUMN('Z2S.GOVERNANCE.PII_LEVEL') = 'LOW' THEN '***' || RIGHT(val, 5)
    WHEN SYSTEM$GET_TAG_ON_CURRENT_COLUMN('Z2S.GOVERNANCE.PII_LEVEL') = 'MEDIUM' THEN '********' || RIGHT(val, 2)
    ELSE HASH(VAL)::VARCHAR
  END;

CREATE OR REPLACE MASKING POLICY Z2S.GOVERNANCE.PHONE_MASK AS (val string) RETURNS string ->
  CASE
    WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN') THEN val
    ELSE 'XXX-XXXX'
  END;

CREATE OR REPLACE MASKING POLICY Z2S.GOVERNANCE.REVENUE_MASK AS (val number) RETURNS number ->
  CASE
    WHEN CURRENT_ROLE() IN ('Z2S_FINANCE_ANALYST', 'ACCOUNTADMIN') THEN val
    WHEN CURRENT_ROLE() IN ('Z2S_HR_MANAGER') THEN ROUND(val, -4) -- Round to nearest 10k
    ELSE NULL
  END;

In [ ]:
-- APPLY POLICY TO TAGS
ALTER TAG Z2S.GOVERNANCE.PII_LEVEL SET MASKING POLICY Z2S.GOVERNANCE.PII_MASK;

-- APPLY POLICY TO INDIVIDUAL COLUMNS
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII ALTER COLUMN PHONE SET MASKING POLICY Z2S.GOVERNANCE.PHONE_MASK;

ALTER TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII ALTER COLUMN CONTACT_PHONE SET MASKING POLICY Z2S.GOVERNANCE.PHONE_MASK;
ALTER TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII ALTER COLUMN ANNUAL_REVENUE SET MASKING POLICY Z2S.GOVERNANCE.REVENUE_MASK;

In [ ]:
CREATE OR REPLACE TABLE Z2S.GOVERNANCE.HR_MANAGER_REGIONS (
    manager_role VARCHAR(50),
    region VARCHAR(20)
);

INSERT INTO Z2S.GOVERNANCE.HR_MANAGER_REGIONS VALUES
('Z2S_HR_MANAGER', 'NORTH'),
('Z2S_HR_MANAGER', 'SOUTH');

CREATE OR REPLACE TABLE Z2S.GOVERNANCE.FINANCE_ANALYST_REGIONS (
    manager_role VARCHAR(50),
    region VARCHAR(20)
);

INSERT INTO Z2S.GOVERNANCE.FINANCE_ANALYST_REGIONS VALUES
('Z2S_FINANCE_ANALYST', 'EAST'),
('Z2S_FINANCE_ANALYST', 'WEST');

In [ ]:
CREATE OR REPLACE ROW ACCESS POLICY Z2S.GOVERNANCE.EMPLOYEE_REGION_POLICY AS (EMPLOYEE_REGION varchar) RETURNS BOOLEAN ->
  CASE
    WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN') THEN TRUE
    WHEN CURRENT_ROLE() = 'Z2S_HR_MANAGER' THEN 
      EXISTS (
        SELECT 1 FROM Z2S.GOVERNANCE.HR_MANAGER_REGIONS 
        WHERE MANAGER_ROLE = CURRENT_ROLE() AND REGION = EMPLOYEE_REGION
      )
    ELSE FALSE
  END;

CREATE OR REPLACE ROW ACCESS POLICY Z2S.GOVERNANCE.CUSTOMER_REGION_POLICY AS (CUSTOMER_REGION varchar) RETURNS BOOLEAN ->
  CASE
    WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN') THEN TRUE
    WHEN CURRENT_ROLE() = 'Z2S_FINANCE_ANALYST' THEN 
      EXISTS (
        SELECT 1 FROM Z2S.GOVERNANCE.FINANCE_ANALYST_REGIONS 
        WHERE MANAGER_ROLE = CURRENT_ROLE() AND region = CUSTOMER_REGION
      )
    ELSE FALSE
  END;

In [ ]:
ALTER TABLE Z2S.DEMO_HR.EMPLOYEES_PII ADD ROW ACCESS POLICY Z2S.GOVERNANCE.EMPLOYEE_REGION_POLICY ON (REGION);

ALTER TABLE Z2S.DEMO_FINANCE.CUSTOMERS_PII ADD ROW ACCESS POLICY Z2S.GOVERNANCE.CUSTOMER_REGION_POLICY ON (REGION);

In [ ]:
USE ROLE ACCOUNTADMIN;
SELECT * FROM Z2S.DEMO_HR.EMPLOYEES_PII;

In [ ]:
USE ROLE Z2S_HR_MANAGER;
SELECT * FROM Z2S.DEMO_HR.EMPLOYEES_PII;

In [ ]:
USE ROLE Z2S_FINANCE_ANALYST;
SELECT * FROM Z2S.DEMO_FINANCE.CUSTOMERS_PII;

In [ ]:
USE ROLE ACCOUNTADMIN;

1. Setup a default warehouse and go through t-shirt sizing
2. Show altering of warehouse sizing
3. Create a multi-cluster warehouse
4. Create a Gen2 Warehouse
5. Connecting a Resource Monitor to a Warehouse

In [ ]:
CREATE OR REPLACE WAREHOUSE Z2S_SIZING_WH
    WAREHOUSE_SIZE = 'X-SMALL'          -- 1 credit/hour - smallest size
    AUTO_SUSPEND = 300                  -- Suspend after 5 minutes of inactivity
    AUTO_RESUME = TRUE                  -- Automatically resume when queries submitted
    INITIALLY_SUSPENDED = TRUE          -- Start in suspended state
    COMMENT = 'Demo warehouse for showing size scaling';

In [ ]:
CREATE OR REPLACE ROLE Z2S_SIZING_WH_U;
CREATE OR REPLACE ROLE Z2S_SIZING_WH_O;

GRANT MONITOR, USAGE  ON WAREHOUSE Z2S_SIZING_WH TO ROLE Z2S_SIZING_WH_U;
GRANT OPERATE, MODIFY ON WAREHOUSE Z2S_SIZING_WH TO ROLE Z2S_SIZING_WH_O;

GRANT ROLE Z2S_SIZING_WH_U TO ROLE Z2S_SIZING_WH_O;

GRANT ROLE Z2S_SIZING_WH_U TO ROLE Z2S_FINANCE_ANALYST;
GRANT ROLE Z2S_SIZING_WH_U TO ROLE Z2S_HR_MANAGER;

In [ ]:
ALTER WAREHOUSE Z2S_SIZING_WH 
SET 
    WAREHOUSE_SIZE = 'LARGE'
    ENABLE_QUERY_ACCELERATION = TRUE;

In [ ]:
SELECT 
    WAREHOUSE_NAME, 
    COUNT(QUERY_ID) AS NUM_ELIGIBLE_QUERIES
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_ACCELERATION_ELIGIBLE
WHERE START_TIME >= DATEADD(MONTH, -1, CURRENT_TIMESTAMP())
    AND END_TIME <= DATEADD(DAY, -1, CURRENT_TIMESTAMP())
GROUP BY WAREHOUSE_NAME
ORDER BY NUM_ELIGIBLE_QUERIES DESC;

In [ ]:
-- Multi-cluster warehouses provide automatic scaling for concurrency
-- Available in Enterprise Edition and higher

-- SCALING_POLICY Options:
-- 1. STANDARD (default): 
--    - Aggressively starts new clusters to minimize query queuing
--    - Prioritizes performance over cost
--    - Starts clusters immediately when queries queue
--    - Best for latency-sensitive workloads
--
-- 2. ECONOMY:
--    - Conserves credits by keeping running clusters fully loaded
--    - Only starts new clusters when existing ones are at capacity
--    - May result in some queuing but reduces costs
--    - Best for batch workloads where slight delays are acceptable
CREATE OR REPLACE WAREHOUSE Z2S_MULTICLUSTER_STANDARD_WH
    WAREHOUSE_SIZE = 'LARGE'
    MIN_CLUSTER_COUNT = 1               -- Always keep 1 cluster running
    MAX_CLUSTER_COUNT = 5               -- Scale up to 5 clusters max
    SCALING_POLICY = 'STANDARD'         -- Minimize queuing by starting clusters quickly
    AUTO_SUSPEND = 60                   -- Suspend after 1 minute (aggressive cost control)
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
    COMMENT = 'Multi-cluster warehouse with STANDARD scaling - prioritizes performance';

In [ ]:
CREATE OR REPLACE WAREHOUSE Z2S_GEN2_WH
    WAREHOUSE_SIZE = 'MEDIUM'
    RESOURCE_CONSTRAINT = 'STANDARD_GEN_2'    -- Specify Gen2 capabilities
    AUTO_SUSPEND = 120                          -- Suspend after 2 minutes
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
    COMMENT = 'Generation 2 standard warehouse with enhanced performance';

In [ ]:
CREATE OR REPLACE RESOURCE MONITOR Z2S_WAREHOUSE_MONITOR
    WITH 
    CREDIT_QUOTA = 100                          -- Monthly credit limit
    FREQUENCY = MONTHLY                         -- Reset quota monthly
    START_TIMESTAMP = IMMEDIATELY               -- Start monitoring now
    TRIGGERS 
        ON 50 PERCENT DO NOTIFY                 -- Send notification at 50% usage
        ON 75 PERCENT DO NOTIFY                 -- Send notification at 75% usage
        ON 90 PERCENT DO SUSPEND                -- Suspend warehouses at 90% (graceful)
        ON 100 PERCENT DO SUSPEND_IMMEDIATE;    -- Immediately suspend at 100%

ALTER WAREHOUSE z2s_sizing_wh SET RESOURCE_MONITOR = Z2S_WAREHOUSE_MONITOR;

In [ ]:
USE WAREHOUSE IDENTIFIER($WAREHOUSE_NAME);

### Caching ###
1. Results Cache: Whenever Snowflake executes a SELECT query, the results returned to the end-user are stored and immediately returned by subsequent queries . While this may seem pointless, it significantly affects query performance, primarily when queries run from dashboards or business intelligence applications where the query text is unchanged. This means (for example) that when a complex query is executed, which can take minutes to complete, subsequent executions return results within milliseconds. Like other Snowflake features, this cache is managed automatically and is cleared whenever the underlying data changes, so users are always given a correct, consistent result. One of the benefits of fetching data from the results cache is that the executing query doesn't need a virtual warehouse. In addition to improving query performance, this can also reduce costs.
2. Metadata Cache: Metadata stored about the table, which includes count, min, and max. This will not engage a virtual warehouse
3. Virtual Warehouse Cache: Each node in the warehouse has dedicated memory and SSD for caching data. This cache is wiped clean when a warehouse is suspended. Data here is aged out on an least recently used(LRU) basis

Criteria for Results Cache
Must match exactly, this includes casing

The query does not include non-reusable functions, which return different results for successive runs of the same query. UUID_STRING, RANDOM, and RANDSTR are good examples of non-reusable functions.

The query does not include external functions.

The query does not select from hybrid tables.

The table data contributing to the query result has not changed.

The persisted result for the previous query is still available.

The role accessing the cached results has the required privileges.

If the query was a SELECT query, the role executing the query must have the necessary access privileges for all the tables used in the cached query.

If the query was a SHOW query, the role executing the query must match the role that generated the cached results.

Any configuration options that affect how the result was produced have not changed.

The table’s micro-partitions have not changed (e.g. been reclustered or consolidated) due to changes to other data in the table.

In [ ]:
SELECT COUNT(*) FROM Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE;

In [ ]:
SELECT * FROM Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE;
SELECT * FROM Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE;
SELECT * FROM Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE rmce;
select * FROM Z2S.DEMO_FINANCE.RESULTS_METADATA_CACHE_EXAMPLE;

Examples of how to do warehouse suspension to benefit from the Warehouse Cache:
- **ETL/ELT and Loading Operations**: Immediate suspension is typically optimal since these workloads don't benefit significantly from cache retention
- **Business Intelligence and SELECT Operations**: Approximately 10-minute suspension delays are cost-optimal to maintain warm caches for end users
- **Development and Data Science Workloads**: Around 5-minute suspension delays provide the best balance

In [ ]:
SELECT REGION, REVENUE FROM Z2S.DEMO_FINANCE.WAREHOUSE_CACHE_EXAMPLE WHERE REVENUE > 2000000;
SELECT REGION, QUARTER, REVENUE FROM Z2S.DEMO_FINANCE.WAREHOUSE_CACHE_EXAMPLE WHERE REVENUE > 2000000 OR REGION = 'Europe';

1. Difference between Information Schema and Account Usage in querying
2. Service Type Consumption Example Query
3. Weekly Usage Example Query

In [ ]:
SELECT 
    SERVICE_TYPE,
    SUM(CREDITS_USED) AS TOTAL_CREDITS,
    ROUND(SUM(CREDITS_USED) / 
        SUM(SUM(CREDITS_USED)) OVER(), 4) * 100 AS PCT_OF_TOTAL_CREDITS,
    AVG(CREDITS_USED) AS AVG_DAILY_CREDITS,
    MAX(CREDITS_USED) AS MAX_DAILY_CREDITS,
    COUNT(DISTINCT usage_date) AS DAYS_WITH_USAGE
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY 
WHERE USAGE_DATE >= DATEADD('day', -30, CURRENT_DATE())
    AND USAGE_DATE < CURRENT_DATE()
GROUP BY SERVICE_TYPE
ORDER BY TOTAL_CREDITS DESC;

In [ ]:
SELECT 
    DATE_TRUNC('WEEK', USAGE_DATE) AS WEEK_START,
    SUM(CREDITS_USED) AS WEEKLY_CREDITS,
    LAG(SUM(CREDITS_USED)) OVER (ORDER BY DATE_TRUNC('WEEK', USAGE_DATE)) AS PREVIOUS_WEEK_CREDITS,
    ROUND((SUM(CREDITS_USED) - LAG(SUM(CREDITS_USED)) OVER (ORDER BY DATE_TRUNC('WEEK', USAGE_DATE))) / 
        NULLIF(LAG(SUM(CREDITS_USED)) OVER (ORDER BY DATE_TRUNC('WEEK', USAGE_DATE)), 0) * 100, 2) AS WEEK_OVER_WEEK_CHANGE_PCT
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY 
WHERE USAGE_DATE >= DATEADD('day', -60, CURRENT_DATE())
    AND USAGE_DATE < CURRENT_DATE()
GROUP BY DATE_TRUNC('WEEK', USAGE_DATE)
ORDER BY WEEK_START DESC;

In [ ]:
-- Roles
DROP ROLE Z2S_SIZING_WH_U;
DROP ROLE Z2S_SIZING_WH_O;
DROP ROLE Z2S_FINANCE_ANALYST;
DROP ROLE Z2S_HR_MANAGER;
DROP DATABASE ROLE Z2S_SCH_GOVERNANCE_R;
DROP DATABASE ROLE Z2S_SCH_GOVERNANCE_W;
DROP DATABASE ROLE Z2S_SCH_GOVERNANCE_C;
DROP DATABASE ROLE Z2S_SCH_FIN_R;
DROP DATABASE ROLE Z2S_SCH_FIN_W;
DROP DATABASE ROLE Z2S_SCH_FIN_C;
DROP DATABASE ROLE Z2S_SCH_HR_R;
DROP DATABASE ROLE Z2S_SCH_HR_W;
DROP DATABASE ROLE Z2S_SCH_HR_C;
DROP DATABASE ROLE Z2S_DB_R;
DROP DATABASE ROLE Z2S_DB_W;
DROP DATABASE ROLE Z2S_DB_C;

-- Database
DROP DATABASE Z2S;

-- Resource Monitors
DROP RESOURCE MONITOR Z2S_WAREHOUSE_MONITOR;

-- Warehouses
DROP WAREHOUSE Z2S_SIZING_WH;
DROP WAREHOUSE Z2S_MULTICLUSTER_STANDARD_WH;
DROP WAREHOUSE Z2S_GEN2_WH;